Here we process Comp-551 A3 datasets. 

In [1]:
! gdown https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0

Downloading...
From: https://drive.google.com/uc?id=1AbxjCBByF-xGV3c7FJ1ztZC1HO0O0uJ0
To: /content/kaggle.json
100% 66.0/66.0 [00:00<00:00, 135kB/s]


In [2]:
! pip install kaggle
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
! kaggle competitions download -c comp-551-fall-2021
! unzip images_l.pkl.zip
! unzip images_test.pkl.zip
! unzip images_ul.pkl.zip
! unzip labels_l.pkl.zip

  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 52.2MB/s]
 88% 87.0M/99.0M [00:00<00:00, 131MB/s]
100% 99.0M/99.0M [00:00<00:00, 169MB/s]
 96% 188M/197M [00:01<00:00, 160MB/s]
100% 197M/197M [00:01<00:00, 167MB/s]
 95% 189M/198M [00:01<00:00, 163MB/s]
100% 198M/198M [00:01<00:00, 175MB/s]
Archive:  images_l.pkl.zip
  inflating: images_l.pkl            
Archive:  images_test.pkl.zip
  inflating: images_test.pkl         
Archive:  images_ul.pkl.zip
  inflating: images_ul.pkl           
Archive:  labels_l.pkl.zip
  inflating: labels_l.pkl            


In [3]:
import io
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU, ZeroPadding2D,Convolution2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import History
from tensorflow.keras.optimizers import Nadam, Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet169
from keras.applications.resnet import ResNet152, ResNet101, ResNet50
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras import layers, regularizers, Input
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau
from keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from google.colab import drive
import h5py
import random

In [4]:
# Image data for training (30,000 sample, each sample is a 56x56 image)
# train_labels = pd.read_csv('train_max_y.csv')
# train_images = pd.read_pickle('train_max_x')
# test_images = pd.read_pickle('test_max_x')
with open("images_l.pkl", 'rb') as f:
    # load into as a numpy array
    train_labelled_data = pickle.load(f)
# Test images. The prediction corresponding to these images should be uploaded. (15,000 samples)
with open("images_test.pkl", 'rb') as f:
    test_data = pickle.load(f)
# Labels for training (30,000 rows, each row is a size 36 binary vector, which is the label to the corresponding image)
with open("labels_l.pkl", 'rb') as f:
    train_labelled_y = pickle.load(f)
# Additional images that can be used for training the classifier.
# Labels for these images are not provided. (30,000 samples, where each sample is a 56x56 image)
with open("images_ul.pkl", 'rb') as f:
    train_unlabelled_x = pickle.load(f)

In [5]:
# download already predicted unlabel train set and load it
! gdown https://drive.google.com/uc?id=1oSJM0BtYu_hiJ7mkSNphJLAmBVnznO4o

Downloading...
From: https://drive.google.com/uc?id=1oSJM0BtYu_hiJ7mkSNphJLAmBVnznO4o
To: /content/ul_train_label.csv
100% 216k/216k [00:00<00:00, 81.5MB/s]


In [6]:
train_unlabelled_y=np.array(pd.DataFrame(pd.read_csv('ul_train_label.csv')))

# clear noise

In [7]:
# data pre processing
def clear_noise (array):
  for n in range (0,array.shape[0]):
    for i in range (0,56):
      for j in range (0,56):
        # pixel of the nth image at position (i,j)
        pixel = array[n][i][j]
        flag = False
        if(i == 0):
        # at the left edge  
          if(j == 0):
          # at the top edge, so only count right and below
            if(array[n][i+1][j]>0 or array[n][i][j+1]>0):
              flag=True
          # at the bottom edge, so only count right and above
          elif(j == 55):
            if(array[n][i+1][j]>0 or array[n][i][j-1]>0):
              flag=True
          else:
          # count right, above and below
            if(array[n][i+1][j]>0 or array[n][i][j+1]>0 or array[n][i][j-1]>0):
              flag=True
        elif(i == 55):
        # at the right edge  
          if(j == 0):
            if(array[n][i-1][j]>0 or array[n][i][j+1]>0):
              flag=True
          elif(j == 55):
            if(array[n][i-1][j]>0 or array[n][i][j-1]>0):
              flag=True
          else:
            if(array[n][i-1][j]>0 or array[n][i][j+1]>0 or array[n][i][j-1]>0):
              flag=True
        else:
          if(j == 0):
            if(array[n][i+1][j]>0 or array[n][i-1][j]>0 or array[n][i][j+1]>0):
              flag=True
          elif(j == 55):
            if(array[n][i+1][j]>0 or array[n][i-1][j]>0 or array[n][i][j-1]>0):
              flag=True
          else:
            if(array[n][i+1][j]>0 or array[n][i-1][j]>0 or array[n][i][j+1]>0 or array[n][i][j-1]>0):
              flag=True   
        if(flag==False):
          # the point is supposed to not link with other point, so this pixel is regarded as noise
          array[n][i][j]=0
        # strengthen the useful pixel, make the background to be dark
        if(array[n][i][j]>70):
          array[n][i][j]=255
        else:
          array[n][i][j]=0
  return array

In [ ]:
train_labelled_data = clear_noise(train_labelled_data)
test_data = clear_noise(test_data)
train_unlabelled_x = clear_noise(train_unlabelled_x)
train_unlabelled_y = clear_noise(train_unlabelled_y)

In [ ]:
train_labelled_data=train_labelled_data.reshape((30000,-1))
print(train_labelled_data.shape)
mydf=pd.DataFrame(train_labelled_data)
mydf.to_csv('processed_train_set.csv',index=False)

test_data=test_data.reshape((15000,-1))
print(test_data.shape)
mydf=pd.DataFrame(test_data)
mydf.to_csv('processed_test_set.csv',index=False)

unlabelled_data=unlabelled_data.reshape((30000,-1))
print(unlabelled_data.shape)
mydf=pd.DataFrame(unlabelled_data)
mydf.to_csv('processed_unlabelled_set_set.csv',index=False)

unlabelled_data_label=unlabelled_data_label.reshape((3000,-1))
print(unlabelled_data_label.shape)
mydf=pd.DataFrame(unlabelled_data_label)
mydf.to_csv('processed_unlabelled_set_label.csv',index=False)

# add noise (noise robustness)

In [8]:
def add_noise(array):
  for n in range (0,array.shape[0]):
    count = 0
    iteration = 0
    while (count<100 and iteration<56*56):
      i = random.randint(0,55)
      j = random.randint(0,55)
      pixel = array[n][i][j]
      if(pixel<50):
        array[n][i][j] = 175
        count = count+1
      iteration = iteration+1
  return array

In [ ]:
train_labelled_data = add_noise(train_labelled_data)
train_unlabelled_x = add_noise(train_unlabelled_x)

In [ ]:
train_labelled_data=train_labelled_data.reshape((30000,-1))
print(train_labelled_data.shape)
mydf=pd.DataFrame(train_labelled_data)
mydf.to_csv('processed_train_set.csv',index=False)

unlabelled_data=unlabelled_data.reshape((30000,-1))
print(unlabelled_data.shape)
mydf=pd.DataFrame(unlabelled_data)
mydf.to_csv('processed_unlabelled_set_set.csv',index=False)